In [10]:
from preprints import identify_preprints
from settings import http_settings

## Month 1st run

Define WRITE_TO_DB and num_to_curate:

* WRITE_TO_DB - flag to write tools to the database (bool)
* num_to_curate - number of high_tools to write to csv file to be manually curated (int or str) ('all')

In [ ]:
WRITE_TO_DB = False
num_to_curate = 100

## Define file paths

In [ ]:
json_preprints = ""
json_publications = ""
json_file = ""
pub2tools_log = ""

## Read pub2tools output

Get tools with high confidence score

In [ ]:
tools = process_tools(json_file)
token = login_prod(http_settings)


In [ ]:
#tools_v = validate_tools(tools, token, http_settings)   # STEP 3. VALIDATE TOOLS FROM PUB2TOOLS OUTPUT

## Separate publications (high and low) and preprints

Identify preprints from validated tools
* rerun (bool):

This function can be used for the first time on to_biotools.json output from Pub2tools
as well as it can be rerun on the preprints json in the future to check for new publications
if there are new publications in low_tools_prp_date.json file, they will be removed from this file
and moved to the publications file

In [ ]:
tools_prp = identify_preprints(rerun = False, tools = tools)

Generate 2 CSVs from _num_to_curate_ first publications

* tools_to_add: tools to add to datase
* tools_left: preprints + publications not in the first _num_to_curate_

In [ ]:
tools_to_add, tools_left = generate_csv_pub(tools_prp, num_to_curate)


## Create json files

Generate 2 json files with tools that will not be curated:

* Preprints (can be used as input for identify_preprints later as rerun = True)
* Publications

In [ ]:
generate_json(tools_left)

## Add tools to curate to dev

In [ ]:
add_tools(tools_to_add, token, http_settings, WRITE_TO_DB)
